In [ ]:
from jax import config
config.update("jax_debug_nans", True)
config.update("jax_enable_x64", True)

from pathlib import Path

from matplotlib import pyplot as plt
import numpy as np

from src.lib_tensors import *
from src.lib_delta import *
from src.delta import *
from src.utils import *
from src.hashable_array import HashableArray

%matplotlib inline

In [ ]:
# master
import sys
sys.path.append('/Users/alexander/GitHub/')
import viperleed
from viperleed.tleedmlib.files import poscar
from viperleed.tleedmlib.files import parameters
from viperleed.tleedmlib.classes.rparams import Rparams
from viperleed.tleedmlib.files.beams import readIVBEAMS
from viperleed.tleedmlib.files.phaseshifts import readPHASESHIFTS
from viperleed.tleedmlib.files.vibrocc import readVIBROCC

In [ ]:
# installable
# import viperleed
# from viperleed.calc.files import poscar
# from viperleed.calc.files import parameters
# from viperleed.calc.classes.rparams import Rparams
# from viperleed.calc.files.beams import readIVBEAMS
# from viperleed.calc.files.phaseshifts import readPHASESHIFTS
# from viperleed.calc.files.vibrocc import readVIBROCC

In [ ]:
data_path = Path('Cu(111)')

In [ ]:
# Read in data from POSCAR and PARAMETERS files
slab = poscar.read(data_path / 'POSCAR')
rparams = parameters.read(data_path / 'PARAMETERS')
parameters.interpret(rparams, slab, silent=False)
slab.full_update(rparams)

In [ ]:
# reading IVBEAMS
rparams.ivbeams = readIVBEAMS(data_path / 'IVBEAMS')
beam_indices = np.array([beam.hk for beam in rparams.ivbeams])

In [ ]:
# reading VIBROCC
readVIBROCC(rparams, slab, data_path / 'VIBROCC')

In [ ]:
# incidence angles
rparams.THETA = 0.0
rparams.PHI = 0.0

In [ ]:
# TODO: add support for energy-dependent LMAX
LMAX = rparams.LMAX.max

In [ ]:
ref_energies = np.linspace(rparams.THEO_ENERGIES.start,
                           rparams.THEO_ENERGIES.stop,
                           rparams.THEO_ENERGIES.n_energies)

In [ ]:
unit_cell_area = np.linalg.norm(np.cross(slab.ab_cell[:,0], slab.ab_cell[:,1]))
# In Bohr radii
unit_cell_area = unit_cell_area / BOHR**2

In [ ]:
# read tensor files for non-bulk atoms
non_bulk_atoms = [at for at in slab.atlist if not at.is_bulk]

In [ ]:
# read phase shifts
phaseshifts_path = data_path / 'PHASESHIFTS'
_, phaseshifts, _, _ = readPHASESHIFTS(slab, rparams, readfile=phaseshifts_path,
                                       check=True, ignoreEnRange=False)


In [ ]:
# interpolate & assign phase shifts to atoms
site_indices = [np.argmax([at.site.isEquivalent(s) for s in slab.sitelist])
                for at in non_bulk_atoms]

#site_indices = np.array([1, 0, 0 ,0 ,0,])

# TODO: with this current implementation, we can not treat chemical
#       pertubations, nor vacancies. We need to implement this.
#       See e.g. iodeltas.generateDeltaInput()
#       (Treating vacancies requires setting zeros for that site)

# shape: (n_energies, n_sites, lmax+1)
interpolated_phaseshifts = interpolate_phaseshifts(
    phaseshifts, LMAX, ref_energies/HARTREE)

# shape: (n_energies, n_atoms, lmax+1)
atom_phaseshifst = interpolated_phaseshifts[:, site_indices, :]

In [ ]:
# assign vibration ampliudes
vibrational_amplitudes = np.array([at.site.vibamp[at.el] for at in non_bulk_atoms])

# TODO: similar to above, this implementation does currently not support
#       changes in vibrational amplitudes, mixed occupations or vacancies.

## Read Tensor files

In [ ]:
read_tensor_num = lambda num: read_tensor(data_path / f'T_{num}', n_beams=len(rparams.ivbeams), n_energies=ref_energies.size, l_max=LMAX+1)
tensors = [read_tensor_num(at.num) for at in non_bulk_atoms]

In [ ]:
delta_amp = lambda displacement: delta_amplitude(LMAX,
                                                 vibrational_amplitudes,
                                                 HashableArray(ref_energies/HARTREE),
                                                 tuple(tensors),
                                                 unit_cell_area,
                                                 HashableArray(atom_phaseshifst),
                                                 displacement)

In [ ]:
np.sum(abs(delta_amp(np.array([[0.0, 0.0, 0.0],]*5))))

In [ ]:
%timeit np.sum(abs(delta_amp(np.array([[0.0, 0.0, 0.0],]*5))))

In [ ]:
# Reference amplitudes should be the same for all tensors
all_ref_amps = [t.ref_amps for t in tensors]
np.sum(np.diff(all_ref_amps, axis=0)) < 1e-10

In [ ]:
# same for v0r
all_v0r = [t.v0r for t in tensors]
np.sum(np.diff(all_v0r, axis=0)) < 1e-10

In [ ]:
# same for v0i
all_v0i = [t.v0i_substrate for t in tensors]
np.sum(np.diff(all_v0i, axis=0)) < 1e-10

In [ ]:
jax.make_jaxpr(delta_amp)(np.array([[0.0, 0.0, 0.0],]*5))

# Intensity

In [ ]:
from src.lib_intensity import *

In [ ]:
# Reference Amplitudes
ref_amps = tensors[0].ref_amps

v_real = tensors[0].v0r
v_imag = tensors[0].v0i_substrate

In [ ]:
# TODO: this needs a better implementation
is_surface_atom = np.array([at.layer.num == 0 for at in non_bulk_atoms])

In [ ]:
lam_prefactor = lambda displacements: intensity_prefactor(ref_energies/HARTREE,
                                                          v_real, v_imag,
                                                         displacements, beam_indices,
                                                         rparams.THETA, rparams.PHI,
                                                         slab.ab_cell, is_surface_atom)

In [ ]:
delta_intensity = lambda displacement: sum_intensity(
    lam_prefactor(displacement), ref_amps, delta_amp(displacement)
)

In [ ]:
plt.figure()
for i in range(-5, 5):
    disp = np.array([[i*0.01, 0.0, 0.0],] +[[0.00, 0.0, 0.0],]*4)
    plt.plot(ref_energies, delta_intensity(disp)[:,0])


In [ ]:
%matplotlib inline
plt.show()

In [ ]:
# Jacobian of intensity seem sto be too expensive to compute

In [ ]:
# # Try jacobian
# disp = np.array([[0.00, 0.0, 0.0],] +[[0.00, 0.0, 0.0],]*4)
# jax.jacfwd(delta_intensity)(disp)[:, 0, 0, 0]

In [ ]:
# plt.figure()
# for i in range(1, 10):
#     disp = np.array([[i*0.01, 0.0, 0.0],] +[[0.00, 0.0, 0.0],]*4)
#     plt.plot(ref_energies, jax.jacfwd(delta_intensity)(disp)[:, 0, 0, 0])

In [ ]:
estimate_function_cost(delta_intensity, disp)

# Interpolation

In [ ]:
from src.interpolation import *

In [ ]:
target_grid = jnp.linspace(ref_energies[0], ref_energies[-1], 200)
interpolator = StaticNotAKnotSplineInterpolator(ref_energies,
                                                target_grid, 3)

In [ ]:
def intensity_interpolated(displacement, beam):
    raw_intensity = delta_intensity(displacement)[:,beam]
    rhs = not_a_knot_rhs(raw_intensity)
    bspline_coeffs = get_bspline_coeffs(interpolator, rhs)
    interpolated_intensity = evaluate_spline(bspline_coeffs, interpolator, 0)
    interpolated_deriv = evaluate_spline(bspline_coeffs, interpolator, 1)
    return interpolated_intensity, interpolated_deriv

In [ ]:
plt.figure()
for i in range(1, 10):
    disp = np.array([[i*0.01, 0.0, 0.0],] +[[0.00, 0.0, 0.0],]*4)
    plt.plot(target_grid*HARTREE, intensity_interpolated(disp,0)[0])
plt.title("Interpolated Intensity")

In [ ]:
plt.figure()
for i in range(1, 10):
    disp = np.array([[i*0.01, 0.0, 0.0],] +[[0.00, 0.0, 0.0],]*4)
    plt.plot(target_grid*HARTREE, intensity_interpolated(disp,0)[1])
plt.title("Interpolated Derivative")

In [ ]:
from src.rfactor import *

In [ ]:
plt.figure()
for i in range(1, 10):
    disp = np.array([[i*0.01, 0.0, 0.0],] +[[0.00, 0.0, 0.0],]*4)
    plt.plot(target_grid*HARTREE, pendry_y(intensity_interpolated(disp,0)[1],intensity_interpolated(disp,0)[0], 4.5))
plt.title("Interpolated Y-function")

# Rfactor

In [ ]:
from src.rfactor import *

In [ ]:
no_displacement = np.array([[0.00, 0.0, 0.0],]*5)
ref_intensity = delta_intensity(no_displacement)[:,0]
R_fun = pendry_R_vs_reference(
    ref_intensity,
    interpolator,
    interpolator,
    4.5,
    3.0,
    0.5,
)

In [ ]:
lam_r = lambda z: jnp.real(R_fun(delta_intensity(
    jnp.asarray([[z, 0.0, 0.0],] +[[0.00, 0.0, 0.0],]*4))[:,0]))

In [ ]:
%timeit lam_r(0.01)

In [ ]:
z_arr = jnp.linspace(-0.05, 0.05, 10)
R_arr = [lam_r(r) for r in z_arr]

In [ ]:
plt.figure()
plt.plot(z_arr, R_arr)

In [ ]:
jax.grad(lam_r)(0.01)

In [ ]:
%timeit jax.grad(lam_r)(0.01)

In [ ]:
R_grad_arr = [jax.grad(lam_r)(r) for r in z_arr]

In [ ]:
plt.figure()
plt.plot(z_arr, R_grad_arr)

In [ ]:
plt.figure()
plt.plot(z_arr, R_grad_arr)
plt.plot(z_arr[:-1], jnp.diff(np.array(R_arr))/ (z_arr[1]-z_arr[0]))

# R2

In [ ]:
ref_intensity_all_beams = delta_intensity(jnp.array([[0.0, 0.0, 0.0],]))

In [ ]:
lam_r2 = lambda z: jnp.real(((delta_intensity(jnp.array([[z, 0.0, 0.0],])) - ref_intensity_all_beams)**2).sum())

In [ ]:
z_arr = jnp.linspace(-0.05, 0.05, 100)
R2_arr = [lam_r2(r) for r in z_arr]
R2_grad_arr = [jax.grad(lam_r2)(r) for r in z_arr]

In [ ]:
plt.figure()
plt.plot(z_arr, R2_arr)

In [ ]:
plt.figure()
plt.plot(z_arr, R2_grad_arr)

# Timing

In [ ]:
# Function cost
estimate_function_cost(lam_r, 0.0)

In [ ]:
# Function cost
estimate_function_cost(lam_r2, 0.0)

In [ ]:
l = jax.jit(lam_r2).lower(0.0).compile()
%timeit l(0.0)

In [ ]:
l2 = jax.jit(jax.grad(lam_r2)).lower(0.0).compile()
%timeit l2(0.0)